In [1]:
from get_dataset import * 
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import models, layers
ds_test = get_dataset("test")
print(next(iter(ds_test.element_spec)))

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-19 11:16:23.083297: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorSpec(shape=(124, 129), dtype=tf.float32, name=None)


2023-04-19 11:16:24.899226: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7444 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
def my_map(x, y):
    return x[..., tf.newaxis], y

def my_filter(x, y):
    return y < 10

ds_train = get_dataset("train").map(my_map).batch(64)
ds_test = get_dataset("test").map(my_map).batch(64)

In [4]:
input_shape = next(iter(ds_train))[0].shape
input_shape

2023-04-19 11:16:30.770070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


TensorShape([64, 124, 129, 1])

In [36]:
model = models.Sequential([
    layers.Input(shape=[124, 129, 1]),
    layers.Resizing(32, 32),
    layers.Conv2D(32, 3, activation='gelu'),
    layers.Conv2D(32, 4, activation='gelu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='gelu'),
    layers.Dropout(0.5),
    layers.Dense(12),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_8 (Resizing)       (None, 32, 32, 1)         0         
                                                                 
 conv2d_20 (Conv2D)          (None, 30, 30, 32)        320       
                                                                 
 conv2d_21 (Conv2D)          (None, 27, 27, 32)        16416     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 dropout_20 (Dropout)        (None, 13, 13, 32)        0         
                                                                 
 flatten_10 (Flatten)        (None, 5408)              0         
                                                     

In [37]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [38]:
EPOCHS = 30
history = model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

Epoch 1/30


2023-04-18 08:51:00.981487: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_10/dropout_20/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


804/804 [==============================] - 4s 3ms/step - loss: 0.9296 - accuracy: 0.7119 - val_loss: 2.2354 - val_accuracy: 0.3515
Epoch 2/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5924 - accuracy: 0.8073 - val_loss: 1.7380 - val_accuracy: 0.4651
Epoch 3/30
804/804 [==============================] - 2s 3ms/step - loss: 0.4789 - accuracy: 0.8438 - val_loss: 1.6283 - val_accuracy: 0.5057
Epoch 4/30
804/804 [==============================] - 2s 3ms/step - loss: 0.4111 - accuracy: 0.8640 - val_loss: 1.3730 - val_accuracy: 0.5547
Epoch 5/30
804/804 [==============================] - 2s 3ms/step - loss: 0.3782 - accuracy: 0.8768 - val_loss: 1.4710 - val_accuracy: 0.5735
Epoch 6/30
804/804 [==============================] - 3s 3ms/step - loss: 0.3446 - accuracy: 0.8859 - val_loss: 1.5195 - val_accuracy: 0.5622
Epoch 7/30
804/804 [==============================] - 2s 3ms/step - loss: 0.3241 - accuracy: 0.8930 - val_loss: 1.4893 - val_accuracy: 0.5881
Epoch 7: early st

In [39]:
y_pred = model.predict(ds_test)
y_pred = tf.argmax(y_pred, axis=1)
y_true = tf.concat(list(ds_test.map(lambda s,lab: lab)), axis=0)
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()


2023-04-18 08:55:27.554697: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Incompatible shapes at component 0: expected [?,61,65,1] but got [64,30,65,1].
	 [[{{node IteratorGetNext}}]]
2023-04-18 08:55:27.554975: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:GPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Incompatible shapes at component 0: expected [?,61,65,1] but got [64,30,65,1].
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


InvalidArgumentError: Graph execution error:

Detected at node 'IteratorGetNext' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/lib/python3.8/asyncio/base_events.py", 